# Учебник по параллельному вызову функций Phi-4 Mini ONNX

Этот ноутбук демонстрирует, как использовать Phi-4 Mini с ONNX Runtime GenAI для параллельного вызова функций. Вызов функций позволяет модели разумно использовать внешние инструменты и API на основе запросов пользователя.

## Обзор

В этом учебнике вы узнаете:
- Как настроить Phi-4 Mini с ONNX Runtime GenAI
- Как определить схемы функций для бронирования авиабилетов и отелей
- Как использовать управляемую генерацию с грамматикой Lark для структурированного вывода
- Как выполнять параллельные вызовы функций для сложных сценариев бронирования путешествий

## Предварительные требования

Перед запуском этого ноутбука убедитесь, что вы:
- Загрузили модель Phi-4 Mini ONNX
- Установили пакет `onnxruntime-genai`
- Имеете базовое понимание концепции вызова функций


## Шаг 1: Импорт необходимых библиотек

Сначала импортируем нужные библиотеки для реализации вызова функций.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Шаг 2: Настройка и конфигурация модели

Теперь мы настроим модель Phi-4 Mini ONNX. Убедитесь, что вы заменили путь на вашу фактическую директорию модели.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Шаг 3: Настройка параметров генерации

Настройте параметры генерации, чтобы управлять поведением вывода модели. Эти настройки обеспечивают детерминированные и целенаправленные ответы для вызова функций.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Шаг 4: Определение доступных функций

Здесь мы определяем функции, которые наш AI-ассистент может вызывать. В этом примере у нас есть две функции, связанные с путешествиями:
1. **booking_flight_tickets**: Для бронирования авиабилетов между аэропортами
2. **booking_hotels**: Для бронирования гостиниц

Определения функций следуют формату схемы вызова функций OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Шаг 5: Вспомогательные функции для генерации грамматики

Эти вспомогательные функции преобразуют определения наших функций в формат грамматики Lark, который используется ONNX Runtime GenAI для управляемой генерации. Это гарантирует, что модель выводит корректные вызовы функций в правильном формате JSON.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Шаг 6: Тестирование генерации грамматики

Давайте протестируем функции генерации грамматики, чтобы увидеть, как они преобразуют наши определения инструментов в нужный формат.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Шаг 7: Подготовка системного запроса и генератора

Теперь мы создадим системный запрос, который информирует модель о доступных инструментах, и настроим генератор с параметрами направленного генерации.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Шаг 8: Инициализация генератора с управляемой генерацией

Теперь мы создадим генератор с нашими настроенными параметрами и применим грамматику Lark для управляемой генерации.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Шаг 9: Тестирование параллельного вызова функций

Теперь протестируем нашу настройку на сложном сценарии бронирования путешествия, который требует вызова нескольких функций.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Генерация вызовов функций

Модель теперь будет генерировать структурированные вызовы функций на основе запроса пользователя. Благодаря управляемой генерации, результат будет представлен в формате JSON, который можно выполнить напрямую.

**Ожидаемый результат**: Модель должна сгенерировать вызовы функций для:
1. `booking_flight_tickets` с деталями рейса из Пекина (PEK) в Париж (CDG)
2. `booking_hotels` с деталями проживания в Париже

Запустите ячейку ниже, чтобы увидеть процесс генерации в реальном времени:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Заключение

🎉 **Поздравляем!** Вы успешно реализовали параллельный вызов функций с использованием Phi-4 Mini и ONNX Runtime GenAI.

### Что вы узнали:

1. **Настройка модели**: Как настроить Phi-4 Mini с ONNX Runtime GenAI
2. **Определение функций**: Как задавать схемы функций для вызова AI-функций
3. **Управляемая генерация**: Как использовать грамматику Lark для структурированной генерации вывода
4. **Параллельные вызовы функций**: Как обрабатывать сложные запросы, требующие нескольких вызовов функций

### Основные преимущества:

- ✅ **Структурированный вывод**: Управляемая генерация обеспечивает корректные JSON-вызовы функций
- ✅ **Параллельная обработка**: Возможность обработки нескольких вызовов функций в одном запросе
- ✅ **Высокая производительность**: ONNX Runtime обеспечивает оптимизированное выполнение
- ✅ **Гибкая схема**: Легкость добавления или изменения определений функций

### Ресурсы:

- [Документация Phi-4 Mini](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [Документация ONNX Runtime GenAI](https://onnxruntime.ai/docs/genai/)
- [Лучшие практики вызова функций](https://platform.openai.com/docs/guides/function-calling)



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
